In [ ]:
# ===============================
# 🎯 Misconceptions Classification - Colab + Drive (Optimized for T4)
# ===============================

import os
import re
import pandas as pd
import numpy as np
import torch
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import KFold
from torch.utils.data import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
)
from sklearn.metrics import average_precision_score


In [ ]:

# ===============================
# 1️⃣ Mount Google Drive
# ===============================
from google.colab import drive
drive.mount('/content/drive')

BASE_FOLD_DIR = '/content/drive/MyDrive/deberta_results_fold'
os.makedirs(BASE_FOLD_DIR, exist_ok=True)

# ===============================
# 2️⃣ Configuration
# ===============================
TRAIN_FILE = "/content/train.csv"  # Update if needed
TEST_FILE = "/content/test.csv"
SUBMISSION_FILE = "/content/drive/MyDrive/submission.csv"

MODEL_NAME = 'microsoft/deberta-v3-base'  # Base model fits T4
MAX_LEN = 512
NUM_EPOCHS = 3
LR = 2e-5
PER_DEVICE_BATCH_SIZE = 8
SEED = 42
N_FOLDS = 5

torch.manual_seed(SEED)
np.random.seed(SEED)
os.environ["WANDB_DISABLED"] = "true"

# ===============================

Mounted at /content/drive


In [ ]:

# 3️⃣ Data Processing
# ===============================
def clean_text(text):
    if isinstance(text, str):
        text = text.lower()
        text = re.sub(r'\s+', ' ', text).strip()
        return text
    return ""

def format_input(row):
    return (
        f"Question: {clean_text(row['QuestionText'])} \n"
        f"Answer: {clean_text(row['MC_Answer'])} \n"
        f"Explanation: {clean_text(row['StudentExplanation'])}"
    )

def load_data(path, is_training=True):
    df = pd.read_csv(path)
    df['input_text'] = df.apply(format_input, axis=1)
    if is_training:
        df['target_label'] = df['Category'].astype(str) + ':' + df['Misconception'].fillna('NA').astype(str)
        df_responses = df.groupby('row_id')['target_label'].apply(list).reset_index(name='labels')
        df_responses = df_responses.merge(df[['row_id', 'input_text']].drop_duplicates(subset=['row_id']), on='row_id')
        return df_responses
    return df

In [ ]:


# ===============================
# 4️⃣ Dataset & Metrics
# ===============================
class MisconceptionDataset(Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        item = {k: v[idx].clone().detach() for k, v in self.encodings.items()}
        if self.labels is not None:
            item['labels'] = torch.tensor(self.labels[idx], dtype=torch.float)
        return item
    def __len__(self):
        return len(self.encodings['input_ids'])

def map_at_k(y_true, y_pred_proba, k=3):
    avg_precisions = []
    sorted_pred_indices = np.argsort(y_pred_proba, axis=1)[:, ::-1]
    for i in range(y_true.shape[0]):
        true_labels = np.where(y_true[i] == 1)[0]
        if len(true_labels) == 0: continue
        top_k_pred_indices = sorted_pred_indices[i, :k]
        running_correct = 0
        total_precision = 0
        remaining_true = set(true_labels)
        for rank, pred_idx in enumerate(top_k_pred_indices, 1):
            if pred_idx in remaining_true:
                running_correct += 1
                total_precision += running_correct / rank
                remaining_true.remove(pred_idx)
                if not remaining_true: break
        if running_correct > 0:
            avg_precisions.append(total_precision / len(true_labels))
    return np.mean(avg_precisions) if avg_precisions else 0.0

def compute_metrics(p, mlb_classes):
    logits = p.predictions
    probabilities = torch.sigmoid(torch.tensor(logits)).numpy()
    y_true = p.label_ids
    return {
        'map3_score': map_at_k(y_true, probabilities, k=3),
        'macro_ap': average_precision_score(y_true, probabilities, average='macro')
    }


In [ ]:

# ===============================
# 5️⃣ K-Fold Training with Drive Save & Resume
# ===============================
def run_kfold_pipeline():
    print("🔹 Loading data...")
    df_responses = load_data(TRAIN_FILE, is_training=True)
    mlb = MultiLabelBinarizer()
    Y_labels = mlb.fit_transform(df_responses['labels'])
    num_labels = len(mlb.classes_)
    print(f"Total labels: {num_labels}")

    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
    kf = KFold(n_splits=N_FOLDS, shuffle=True, random_state=SEED)

    for fold, (train_idx, val_idx) in enumerate(kf.split(df_responses)):
        fold_dir = os.path.join(BASE_FOLD_DIR, f'fold_{fold}')
        os.makedirs(fold_dir, exist_ok=True)

        if os.path.exists(os.path.join(fold_dir, "completed.txt")):
            print(f"✅ Fold {fold} already trained. Skipping...")
            continue

        print(f"\n▶️ Starting Fold {fold}...")
        fold_train_df = df_responses.iloc[train_idx].reset_index(drop=True)
        fold_val_df = df_responses.iloc[val_idx].reset_index(drop=True)

        X_train, Y_train = fold_train_df['input_text'].tolist(), Y_labels[train_idx]
        X_val, Y_val = fold_val_df['input_text'].tolist(), Y_labels[val_idx]

        train_encodings = tokenizer(X_train, truncation=True, padding='max_length', max_length=MAX_LEN, return_tensors='pt')
        val_encodings = tokenizer(X_val, truncation=True, padding='max_length', max_length=MAX_LEN, return_tensors='pt')

        train_dataset = MisconceptionDataset(train_encodings, Y_train)
        val_dataset = MisconceptionDataset(val_encodings, Y_val)
        data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

        model = AutoModelForSequenceClassification.from_pretrained(
            MODEL_NAME, num_labels=num_labels, problem_type="multi_label_classification"
        )
        # model.gradient_checkpointing_enable()  # save memory

        training_args = TrainingArguments(
            output_dir=fold_dir,
            num_train_epochs=NUM_EPOCHS,
            per_device_train_batch_size=PER_DEVICE_BATCH_SIZE,
            per_device_eval_batch_size=PER_DEVICE_BATCH_SIZE*2,
            learning_rate=LR,
            warmup_ratio=0.1,
            weight_decay=0.01,
            logging_steps=50,
            save_strategy="epoch",
            eval_strategy="epoch",
            load_best_model_at_end=True,
            metric_for_best_model='map3_score',
            greater_is_better=True,
            fp16=True,
            save_total_limit=2,
            report_to="none"
        )

        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=train_dataset,
            eval_dataset=val_dataset,
            tokenizer=tokenizer,
            compute_metrics=lambda p: compute_metrics(p, mlb.classes_),
            data_collator=data_collator
        )

        trainer.train()

        val_preds = trainer.predict(val_dataset)
        val_map3 = map_at_k(Y_val, torch.sigmoid(torch.tensor(val_preds.predictions)).numpy(), k=3)
        print(f"✅ Fold {fold} MAP@3: {val_map3:.4f}")

        # mark fold as completed
        with open(os.path.join(fold_dir, "completed.txt"), "w") as f:
            f.write(f"done\nMAP@3: {val_map3:.4f}\n")

        # save tokenizer
        tokenizer.save_pretrained(fold_dir)

        del model, trainer, train_dataset, val_dataset
        torch.cuda.empty_cache()

    print("\n🎉 All folds trained successfully!")

# ===============================
if __name__ == "__main__":
    run_kfold_pipeline()


🔹 Loading data...
Total labels: 65


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(



▶️ Starting Fold 0...


pytorch_model.bin:   0%|          | 0.00/371M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/371M [00:00<?, ?B/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-550114123.py:61: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 2, 'bos_token_id': 1}.
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
 

In [ ]:

# ===============================
# 6️⃣ Final Ensemble Prediction
# ===============================
def ensemble_and_predict():
    print("--- Generating ensemble predictions ---")
    df_test = load_data(TEST_FILE, is_training=False)
    X_test = df_test['input_text'].tolist()
    test_dataset = None
    all_test_preds = []
    tokenizer = AutoTokenizer.from_pretrained(BASE_FOLD_DIR + '/fold_0', use_fast=True)

    test_encodings = tokenizer(X_test, truncation=True, padding='max_length', max_length=MAX_LEN, return_tensors='pt')
    test_dataset = MisconceptionDataset(test_encodings)

    for fold in range(N_FOLDS):
        fold_dir = os.path.join(BASE_FOLD_DIR, f'fold_{fold}')
        if not os.path.exists(os.path.join(fold_dir, "pytorch_model.bin")):
            print(f"Skipping fold {fold} — model not found.")
            continue

        model = AutoModelForSequenceClassification.from_pretrained(fold_dir)
        trainer = Trainer(model=model, tokenizer=tokenizer)
        raw_predictions = trainer.predict(test_dataset).predictions
        probabilities = torch.sigmoid(torch.tensor(raw_predictions)).numpy()
        all_test_preds.append(probabilities)
        del model, trainer
        torch.cuda.empty_cache()

    avg_probabilities = np.mean(all_test_preds, axis=0)
    top_3_indices = np.argsort(avg_probabilities, axis=1)[:, ::-1][:, :3]

    # Get MultiLabelBinarizer classes from first fold
    mlb = MultiLabelBinarizer()
    mlb.fit_transform([[]])  # Placeholder
    mlb.classes_ = pd.read_csv(TRAIN_FILE)['Category'].astype(str) + ':' + pd.read_csv(TRAIN_FILE)['Misconception'].fillna('NA').astype(str)
    mlb.classes_ = np.unique(mlb.classes_)

    predictions = [' '.join([mlb.classes_[i] for i in row]) for row in top_3_indices]

    submission_df = pd.DataFrame({
        'row_id': df_test['row_id'],
        'Category:Misconception': predictions
    })
    submission_df.to_csv(SUBMISSION_FILE, index=False)
    print(f"✅ Submission saved to {SUBMISSION_FILE}")
    print(submission_df.head())

# ===============================
# 7️⃣ Execute
# ===============================
if __name__ == "__main__":
    ensemble_and_predict()